In [1]:
-- Create Database
CREATE DATABASE AccountingSystem;
GO

USE AccountingSystem;
GO


CREATE TABLE ChartOfAccounts (
    AccountId INT PRIMARY KEY IDENTITY,
    AccountName NVARCHAR(100),
    ParentId INT NULL,
    AccountType NVARCHAR(50)
);

CREATE TABLE Vouchers (
    VoucherId INT PRIMARY KEY IDENTITY,
    VoucherType NVARCHAR(50),
    ReferenceNo NVARCHAR(50),
    VoucherDate DATE
);

CREATE TABLE VoucherEntries (
    EntryId INT PRIMARY KEY IDENTITY,
    VoucherId INT,
    AccountId INT,
    Debit DECIMAL(18,2),
    Credit DECIMAL(18,2),
    FOREIGN KEY (VoucherId) REFERENCES Vouchers(VoucherId),
    FOREIGN KEY (AccountId) REFERENCES ChartOfAccounts(AccountId)
);

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:01.103

In [2]:
CREATE TYPE VoucherEntryType AS TABLE (
    AccountId INT,
    Debit DECIMAL(18,2),
    Credit DECIMAL(18,2)
);

Commands completed successfully.

Total execution time: 00:00:00.020

In [3]:
CREATE PROCEDURE sp_ManageChartOfAccounts
    @Action NVARCHAR(10),
    @AccountId INT = NULL,
    @AccountName NVARCHAR(100) = NULL,
    @ParentId INT = NULL,
    @AccountType NVARCHAR(50) = NULL
AS
BEGIN
    IF @Action = 'INSERT'
        INSERT INTO ChartOfAccounts (AccountName, ParentId, AccountType)
        VALUES (@AccountName, @ParentId, @AccountType);

    ELSE IF @Action = 'UPDATE'
        UPDATE ChartOfAccounts
        SET AccountName = @AccountName, ParentId = @ParentId, AccountType = @AccountType
        WHERE AccountId = @AccountId;

    ELSE IF @Action = 'DELETE'
        DELETE FROM ChartOfAccounts WHERE AccountId = @AccountId;

    ELSE IF @Action = 'SELECT'
        SELECT * FROM ChartOfAccounts;
END;

GO

CREATE PROCEDURE sp_SaveVoucher
    @VoucherType NVARCHAR(50),
    @ReferenceNo NVARCHAR(50),
    @VoucherDate DATE,
    @Entries VoucherEntryType READONLY
AS
BEGIN
    DECLARE @VoucherId INT;
    INSERT INTO Vouchers (VoucherType, ReferenceNo, VoucherDate)
    VALUES (@VoucherType, @ReferenceNo, @VoucherDate);
    SET @VoucherId = SCOPE_IDENTITY();

    INSERT INTO VoucherEntries (VoucherId, AccountId, Debit, Credit)
    SELECT @VoucherId, AccountId, Debit, Credit FROM @Entries;
END;
GO

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.027

In [4]:
CREATE LOGIN appUserLogin WITH PASSWORD = 'sf123456!', CHECK_POLICY = ON;
GO

: Msg 15025, Level 16, State 1, Line 1
The server principal 'appUserLogin' already exists.

Total execution time: 00:00:00.044

In [5]:
USE AccountingSystem;
GO

CREATE USER appUser FOR LOGIN appUserLogin;
GO

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.021

In [6]:
USE AccountingSystem;
GO

-- Grant permissions to create tables
GRANT CREATE TABLE TO appUser;
GO

-- Grant permissions to alter the dbo schema (essential for EF Core migrations)
GRANT ALTER ON SCHEMA::dbo TO appUser;
GO

-- Grant REFERENCES permission on the dbo schema (essential for foreign key constraints)
GRANT REFERENCES ON SCHEMA::dbo TO appUser;
GO

-- Grant DML operations (SELECT, INSERT, UPDATE, DELETE) on all objects within the dbo schema
GRANT SELECT, INSERT, UPDATE, DELETE ON SCHEMA::dbo TO appUser;
GO

-- Alternatively, grant permissions on specific tables if they already exist:
-- GRANT SELECT, INSERT, UPDATE, DELETE ON YourTableName TO appUser;
-- GO

-- Grant permission to execute stored procedures (if you use them)
GRANT EXECUTE ON SCHEMA::dbo TO appUser;
GO

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.035

In [10]:
-- Seed base accounts
INSERT INTO ChartOfAccounts (AccountName, ParentId, AccountType) VALUES
('Assets', NULL, 'Group'),
('Cash', 1, 'Ledger'),
('Bank', 1, 'Ledger'),
('Liabilities', NULL, 'Group'),
('Accounts Payable', 4, 'Ledger'),
('Equity', NULL, 'Group'),
('Owner Capital', 6, 'Ledger'),
('Revenue', NULL, 'Group'),
('Sales', 8, 'Ledger'),
('Expenses', NULL, 'Group'),
('Rent Expense', 10, 'Ledger'),
('Supplies Expense', 10, 'Ledger');


(12 rows affected)

Total execution time: 00:00:00.060

In [11]:
EXEC sp_ManageChartOfAccounts @Action = 'SELECT'


(12 rows affected)

Total execution time: 00:00:00.097

AccountId,AccountName,ParentId,AccountType
1,Assets,NULL,Group
2,Cash,1,Ledger
3,Bank,1,Ledger
4,Liabilities,NULL,Group
5,Accounts Payable,4,Ledger
6,Equity,NULL,Group
7,Owner Capital,6,Ledger
8,Revenue,NULL,Group
9,Sales,8,Ledger
10,Expenses,NULL,Group


In [14]:
ALTER PROCEDURE sp_ManageChartOfAccounts
    @Action NVARCHAR(20),
    @AccountId INT = NULL,
    @AccountName NVARCHAR(100) = NULL,
    @ParentId INT = NULL,
    @AccountType NVARCHAR(50) = NULL
AS
BEGIN
    SET NOCOUNT ON;

    IF @Action = 'SELECT'
    BEGIN
        SELECT AccountId, AccountName, ParentId, AccountType FROM ChartOfAccounts ORDER BY ParentId, AccountName
    END

    ELSE IF @Action = 'INSERT'
    BEGIN
        INSERT INTO ChartOfAccounts (AccountName, ParentId, AccountType)
        VALUES (@AccountName, @ParentId, @AccountType)
    END

    ELSE IF @Action = 'UPDATE'
    BEGIN
        UPDATE ChartOfAccounts
        SET AccountName = @AccountName,
            ParentId = @ParentId,
            AccountType = @AccountType
        WHERE AccountId = @AccountId
    END

    ELSE IF @Action = 'DELETE'
    BEGIN
        DELETE FROM ChartOfAccounts WHERE AccountId = @AccountId
    END
END

Commands completed successfully.

Total execution time: 00:00:00.082

In [20]:
ALTER PROCEDURE sp_GetVouchers
AS
BEGIN
    SELECT VoucherId, VoucherType, ReferenceNo, VoucherDate FROM Vouchers ORDER BY VoucherDate DESC;
END
GO

ALTER PROCEDURE sp_GetVoucherEntries
    @VoucherId INT
AS
BEGIN
    SELECT ve.VoucherId, ve.AccountId, ve.Debit, ve.Credit, coa.AccountName
    FROM VoucherEntries ve
    INNER JOIN ChartOfAccounts coa ON ve.AccountId = coa.AccountId
    WHERE ve.VoucherId = @VoucherId
    ORDER BY ve.AccountId;
END
GO



Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.015

In [21]:
EXEC sp_GetVouchers;


(2 rows affected)

Total execution time: 00:00:00.107

VoucherId,VoucherType,ReferenceNo,VoucherDate
1,Journal,REF123,2025-06-27
2,Payment,REF124,2025-06-27
